Sarcasm Detection Code - Method 1

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install datasets
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from datasets import load_dataset

from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

import numpy as np
from datasets import load_metric

from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

npsarcasm = load_dataset('csv',data_files={'train': '/content/drive/MyDrive/SarcasmProjectNLP/npsarcasm_train.csv', 
                                           'test': '/content/drive/MyDrive/SarcasmProjectNLP/npsarcasm_test.csv'})

# sarcasm = load_dataset('csv',data_files={'train': '/content/drive/MyDrive/SarcasmProjectNLP/sarcasm_train.csv', 
#                                           'test': '/content/drive/MyDrive/SarcasmProjectNLP/sarcasm_test.csv'})

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/1c4513b2eedbda136f57676a34eea67aba266e5c/config.json
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.25.1",
  "vocab_size": 30522
}

loading file vocab.txt from cache at /root/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/1c4513b2eedbda136f57676a34eea67aba266e5c/vocab.txt
loading file tokenizer.json from cache at /root/.cache/huggingface/hub/models--distilbert-base-uncased/snapsh

Extracting data files #0:   0%|          | 0/1 [00:00<?, ?obj/s]

Extracting data files #1:   0%|          | 0/1 [00:00<?, ?obj/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-b3cd6335f3e06973/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
# initiate tokenizing function
def preprocess_function(examples):
  return tokenizer(examples["text"], truncation=True)

In [ ]:
#tokenize the train and test sets
tokenized_train = sarcasm["train"].map(preprocess_function)
tokenized_test = sarcasm["test"].map(preprocess_function)

  0%|          | 0/40000 [00:00<?, ?ex/s]

  0%|          | 0/10000 [00:00<?, ?ex/s]

In [ ]:
# initiate the function for computing metrics, namely with precision, recall and f1
def compute_metrics(eval_pred):
  load_f1 = load_metric("f1")
  load_precision = load_metric("precision")
  load_recall = load_metric("recall")
  
  logits, labels = eval_pred
  predictions = np.argmax(logits, axis=-1)
  precision = load_precision.compute(predictions=predictions, references=labels)["precision"]
  recall = load_recall.compute(predictions=predictions, references=labels)["recall"]
  f1 = load_f1.compute(predictions=predictions, references=labels)["f1"]
  return {"precision": precision, "recall": recall, "f1": f1}

In [ ]:
# Tried a variety of epochs, 4 was determined to give the best results

training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/SarcasmProjectNLP/results",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    # num_train_epochs=3,
    num_train_epochs=4,
    # num_train_epochs=5,
    # num_train_epochs=6,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 40000
  Num Epochs = 4
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 10000
  Number of trainable parameters = 66955010
You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster

Step,Training Loss
500,0.619600
1000,0.578500
1500,0.558900
2000,0.556500
2500,0.543800
3000,0.471900
3500,0.465500
4000,0.459100
4500,0.460500
5000,0.454600


Saving model checkpoint to /content/drive/MyDrive/SarcasmProjectNLP/results2/checkpoint-500
Configuration saved in /content/drive/MyDrive/SarcasmProjectNLP/results2/checkpoint-500/config.json
Model weights saved in /content/drive/MyDrive/SarcasmProjectNLP/results2/checkpoint-500/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/SarcasmProjectNLP/results2/checkpoint-500/tokenizer_config.json
Special tokens file saved in /content/drive/MyDrive/SarcasmProjectNLP/results2/checkpoint-500/special_tokens_map.json
Saving model checkpoint to /content/drive/MyDrive/SarcasmProjectNLP/results2/checkpoint-1000
Configuration saved in /content/drive/MyDrive/SarcasmProjectNLP/results2/checkpoint-1000/config.json
Model weights saved in /content/drive/MyDrive/SarcasmProjectNLP/results2/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/SarcasmProjectNLP/results2/checkpoint-1000/tokenizer_config.json
Special tokens file saved in /content/drive/My

TrainOutput(global_step=10000, training_loss=0.4050391876220703, metrics={'train_runtime': 1075.1325, 'train_samples_per_second': 148.819, 'train_steps_per_second': 9.301, 'total_flos': 1626819704075712.0, 'train_loss': 0.4050391876220703, 'epoch': 4.0})

In [ ]:
trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 10000
  Batch size = 16


{'eval_loss': 0.8011631965637207,
 'eval_precision': 0.7318039624608967,
 'eval_recall': 0.7019403880776155,
 'eval_f1': 0.7165611598938124,
 'eval_runtime': 16.499,
 'eval_samples_per_second': 606.096,
 'eval_steps_per_second': 37.881,
 'epoch': 4.0}